In [1]:
import requests
import pandas as pd
import numpy as np
import urllib.request

In [8]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])

# downloads all the latest photos.
# traffic_image_df.apply(lambda x: urllib.request.urlretrieve(x['ImageLink'],'%s.jpg'%x['CameraID']) ,axis=1)

In [10]:
traffic_image_df.loc[0,'ImageLink']

'https://dm-traffic-camera-itsc.s3.ap-southeast-1.amazonaws.com/2022-10-27/22-40/1001_2238_20221027224012_485249.jpg?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEC0aDmFwLXNvdXRoZWFzdC0xIkgwRgIhAKMJCjmpUypnDP12MeMQo680kC%2F9KkHWa3ibvwH7bnWAAiEAvhAytxiwSMEvYT4o1SLbl0xDR60ji7R2aU4F7EXvLD8q2wQIFhADGgwzNDA2NDUzODEzMDQiDEzz3nlbqPOeeO0bIyq4BOVCGbAF%2FeQovysIqhGrA5hKUX%2FrcBIsmzUNGcNXSCfogF1XxJ6ZarmO4R%2FdwzttjMbOEH62%2B5i%2Bf3b2Jkp6e03gc61G35bnAX1HHh92X1Olz7dFE9czaY9AOeUFVor%2F0Nn7mw5CZ5vKMOEQ2oX%2FqDFTWe65RhujPAQyuPKhy3fy%2FwIbLa%2Ff0dDZ0S63fnfSojIFtSXCB3%2ByafYAupPz7VvT%2FI2sO9h5w4D3J4%2B%2BSRDsSOGXVcE77575%2B7dVrzTXP6CiGMaGsIgoj1GBlAwqOoUsGCt2lqHsDlflCLUZhLQ%2BKtPynApAuHh3B8VqTP1nWpvV7IA%2Bzuie9XxPRaGfsZhAS4fShIqjwxoznudnyN%2BelUhFNHQVneR2ejXEPLbmq2NdR000dNGxtiTjfw%2BwXdzHbklzJkrjx2J1Qyf%2FHXKh3ubEYLKwTptk%2F5sJwjWzFLZQG7ApUYP51%2FGO30pC6n%2Fi4ARRqBKsSqoSCVuY7PXGLhWI7CXRJCm4ILBwCEVAl7%2FTTnXNFN9CR5YriLmF0OaeE7Wmn5uBuZdPrhe8u%2BdsZskyA1cwEGQb68XWPaloB7jU81fM2Szio7mUafQoa5ox9SiZzk0ovuBQoxo1tTSpqJho

In [3]:
traffic_speed_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficSpeedBandsv2'
traffic_speed_req=requests.get(url=traffic_speed_url,headers=headers_val)
traffic_speed_df=pd.DataFrame(eval(traffic_speed_req.content)['value'])

# road category 
# A – Expressways
# B – Major Arterial Roads
# C – Arterial Roads
# D – Minor Arterial Roads
# E – Small Roads
# F – Slip Roads
# G – No category info available

# speed bands
# 1 – indicates speed range from 0 < 9
# 2 – indicates speed range from 10 < 19
# 3 – indicates speed range from 20 < 29
# 4 – indicates speed range from 30 < 39
# 5 – indicates speed range from 40 < 49
# 6 – indicates speed range from 50 < 59
# 7 – indicates speed range from 60 < 69
# 8 – speed range from 70 or more


In [4]:
def location_splitter(location):
    location=location.split(' ')
    return location[0],location[1],location[2],location[3]

In [5]:
traffic_speed_df['start_latitude'],traffic_speed_df['start_longitude'],traffic_speed_df['end_latitude'],traffic_speed_df['end_longitude']=np.vectorize(location_splitter)(traffic_speed_df['Location'])

In [6]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])


# NEA API
For some reason this one need to constantly call and keep track of previous results. Sometimes calls does not return all data.
Some names are given as S123, etc, looking to try OneMap reverse geocoding to get road name from lat long.

In [7]:
# weather_df=pd.read_csv('weather.csv')
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')

new_weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

new_weather_df['latitude']=new_weather_df['location'].apply(lambda x: x['latitude'])
new_weather_df['longitude']=new_weather_df['location'].apply(lambda x: x['longitude'])
new_weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
new_weather_df['timestamp']=pd.to_datetime(new_weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall in mm'})

new_weather_df=new_weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
new_weather_df=new_weather_df.drop(['id','device_id','station_id','location'],axis=1)

# pd.concat([new_weather_df,weather_df]).drop_duplicates().to_csv('weather.csv',index=False)

In [46]:
new_weather_df

,name,latitude,longitude,timestamp,rainfall in mm
0,Alexandra Road,1.29370,103.81250,2022-10-24 21:45:00+08:00,0
1,Ang Mo Kio Avenue 5,1.37640,103.84920,2022-10-24 21:45:00+08:00,0
2,Bukit Panjang Road,1.38240,103.76030,2022-10-24 21:45:00+08:00,0
3,Bukit Timah Road,1.31910,103.81910,2022-10-24 21:45:00+08:00,0
4,Chai Chee Street,1.32300,103.92170,2022-10-24 21:45:00+08:00,0
...,...,...,...,...,...
65,Upper Serangoon Road,1.33820,103.86570,2022-10-24 21:45:00+08:00,0
66,Upper Thomson Road,1.37010,103.82710,2022-10-24 21:45:00+08:00,0
67,West Coast Highway,1.28100,103.75400,2022-10-24 21:45:00+08:00,0
68,Woodlands Avenue 9,1.44387,103.78538,2022-10-24 21:45:00+08:00,0


# Onemap (to get road name from lat long (WIP)

In [47]:
onemapTokenAPIResponse=requests.post('https://developers.onemap.sg/privateapi/auth/post/getToken',json={'email':'leejin@u.nus.edu','password':'Whysohardtochange123!'})#.content
onemapAPItoken=eval(onemapTokenAPIResponse.content)['access_token']

In [48]:
def roadnamegrabber(name,latitude,longitude):
    if name[0]=='S' and len(name)<5:
        tempResponse=requests.get('https://developers.onemap.sg/privateapi/commonsvc/revgeocode?location=%s,%s&token=%s&buffer=100&addressType=all'%(str(latitude),str(longitude),onemapAPItoken))
        name=pd.DataFrame(eval(tempResponse.content)['GeocodeInfo'])['ROAD'].mode()[0]
    return name

In [49]:
new_weather_df['name']=np.vectorize(roadnamegrabber)(new_weather_df['name'],new_weather_df['latitude'],new_weather_df['longitude'])

In [50]:
# Downloads all the csv files
traffic_image_df.to_csv('traffic_image.csv',index=False)
traffic_speed_df.to_csv('traffic_speed.csv',index=False)
traffic_incidents_df.to_csv('traffic_incidents.csv',index=False)
new_weather_df.to_csv('weather.csv',index=False)

# END